In [38]:
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import itertools
import matplotlib.pyplot as plt

t_file = "trump_1811_1604410542"
b_file = "biden_1817_1604410734"

df_t = pd.read_csv("../data/processed/"+t_file+"_exploded.csv")
df_b = pd.read_csv("../data/processed/"+b_file+"_exploded.csv")

df_t["file"]="trump"
df_b["file"]="biden"

data = pd.concat([df_t,df_b])
data["file"].value_counts()

biden    12793
trump    12510
Name: file, dtype: int64

In [45]:
M = nx.MultiGraph()

M.add_nodes_from(data["hashtags"].unique(),typ="hashtag",bipartite=0)
M.add_nodes_from(data["uniqueId"].unique(),typ="user",bipartite=1)

edge_data_t = data[data["file"]=="trump"]
M.add_edges_from(list(zip(edge_data_t["hashtags"],edge_data_t["uniqueId"])),file="trump",weight=1)

edge_data_b = data[data["file"]=="biden"]
M.add_edges_from(list(zip(edge_data_b["hashtags"],edge_data_b["uniqueId"])),file="biden",weight=1)
print(nx.info(M))

Name: 
Type: MultiGraph
Number of nodes: 5842
Number of edges: 25303
Average degree:   8.6624


In [52]:
G = nx.Graph()
for u,v,data in M.edges(data=True):
    w = data['weight'] if 'weight' in data else 1.0
    if G.has_edge(u,v):
        G[u][v]['weight'] += w
    else:
        G.add_node(u,typ="hashtag")
        G.add_node(v,typ="user")
        G.add_edge(u, v, weight=w)

In [60]:
remove = [node for node,degree in dict(G.degree()).items() if degree < 2]
G.remove_nodes_from(remove)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 3262
Number of edges: 15134
Average degree:   9.2790


In [59]:
nx.write_gexf(G,"../data/graphs/graph.gexf")